# 列表渲染

### 使用 v-for 遍历 数组 生成元素

v-for 指令需要限定格式为 item in items 的特殊语法，其中，items 是原始数据数组(source data array)，而 item 是数组中每个迭代元素的指代别名(alias)：
```js
var example1 = new Vue({
  el: '#example-1',
  data: {
    items: [
      { message: 'Foo' },
      { message: 'Bar' }
    ]
  }
})
```

```html
<ul id="example-1">
  <li v-for="item in items">
    {{ item.message }}
  </li>
</ul>
```

v-for 还支持可选的第二个参数，作为当前项的索引。
```html
<ul id="example-2">
  <li v-for="(item, index) in items">
    {{ parentMessage }} - {{ index }} - {{ item.message }}
  </li>
</ul>
```
可以不使用 in，而是使用 of 作为分隔符，因为它更加接近 JavaScript 迭代器语法：
`<div v-for="item of items"></div>`

### 使用 v-for 遍历 对象 生成元素

可以使用 v-for 来遍历对象的属性。

```html
<ul id="v-for-object" class="demo">
  <li v-for="value in object">
    {{ value }}
  </li>
</ul>
```
```js
new Vue({
  el: '#v-for-object',
  data: {
    object: {
      firstName: 'John',
      lastName: 'Doe',
      age: 30
    }
  }
})
```

还可以提供第二个参数，作为对象的键名(key)：
```html
<div v-for="(value, key) in object">
  {{ key }}: {{ value }}
</div>
```

还可以第三个参数作为索引(index)：
```html
<div v-for="(value, key, index) in object">
  {{ index }}. {{ key }}: {{ value }}
</div>
```

**注意:** 在遍历一个对象时，是按照 `Object.keys()` 得出 key 的枚举顺序来遍历，无法保证在所有 JavaScript 引擎实现中完全一致。

### 关于key

为了便于 Vue 跟踪每个节点的身份，从而重新复用(reuse)和重新排序(reorder)现有元素，你需要为每项提供唯一的 key 属性，从而给 Vue 一个提示。

理想的 key 值是每项都有唯一的 id。这个特殊属性和 Vue 1.x 中的 track-by 粗略相同，但是它会类似于属性运行，所以你需要使用 v-bind 将其与动态值绑定在一起（这里使用简写）：

```html
<div v-for="item in items" :key="item.id">
  <!-- content -->
</div>
```

推荐，在使用 v-for 时，尽可能提供一个 key，除非迭代的 DOM 内容足够简单，或者你是故意依赖于默认行为来获得性能提升。

### 数组变化检测(Array Change Detection)

Vue 将观察数组(observed array)的变化数组方法(mutation method)包裹起来，以便在调用这些方法时，也能够触发视图更新。这些包裹的方法如下：

* push()
* pop()
* shift()
* unshift()
* splice()
* sort()
* reverse()

可以打开控制台，然后对前面示例中的 items 数组调用变化数组方法。例如：`example1.items.push({ message: 'Baz' })`。

变化数组方法(mutation method)，在调用后会改变原始数组。

非变化数组方法(non-mutating method)，例如 filter(), concat() 和 slice()，这些方法都不会直接修改操作原始数组，而是**返回一个新数组**。

例如:
```js
example1.items = example1.items.filter(function (item) {
  return item.message.match(/Foo/)
})
```
你可能会认为这将导致 Vue 丢弃现有 DOM 并重新渲染(re-render)整个列表, 但是情况并非如此。Vue 实现了一些智能启发式方法(smart heuristic)来最大化 DOM 元素重用(reuse)，因此将一个数组替换为包含重叠对象的另一个数组，会是一种非常高效的操作。


### 注意事项

由于 JavaScript 的限制，Vue 无法检测到以下数组变动：

1. 当你使用索引直接设置一项时，例如 `vm.items[indexOfItem] = newValue`
2. 当你修改数组长度时，例如 `vm.items.length = newLength`

例如:
```js
var vm = new Vue({
  data: {
    items: ['a', 'b', 'c']
  }
})
vm.items[1] = 'x' // 不是响应的
vm.items.length = 2 // 不是响应的
```

解决第 1 个问题:
```js
// Vue.set
Vue.set(vm.items, indexOfItem, newValue)
// 还可以使用 vm.$set 实例方法，这也是全局 Vue.set 方法的别名：
vm.$set(vm.items, indexOfItem, newValue)
```

解决第 2 个问题，你可以使用 splice:
`vm.items.splice(newLength)`

### 对象变化检测说明(Object Change Detection Caveats)

再次申明，受现代 Javascript 的限制， Vue 无法检测到对象属性的添加或删除。例如：
```js
var vm = new Vue({
  data: {
    a: 1
  }
})
// `vm.a` 是响应的

vm.b = 2
// `vm.b` 不是响应的
```

Vue 不允许在已经创建的实例上，动态地添加新的**根级响应式属性(root-level reactive property)**。然而，可以使用 `Vue.set(object, key, value) `方法，将响应式属性添加到嵌套的对象上。例如，给出：

```js
var vm = new Vue({
  data: {
    userProfile: {
      name: 'Anika'
    }
  }
})
```

可以向嵌套的 userProfile 对象，添加一个新的 age 属性：

`Vue.set(vm.userProfile, 'age', 27)`
还可以使用 vm.$set 实例方法，这也是全局 Vue.set 方法的别名：
`vm.$set(vm.userProfile, 'age', 27)`

有时，你想要向**已经存在的对象上添加一些新的属性**，例如使用 `Object.assign()` 或 `_.extend()` 方法。

在这种情况下，**应该创建一个新的对象**，这个对象同时具有两个对象的所有属性，因此，改为：

```js
Object.assign(vm.userProfile, {
  age: 27,
  favoriteColor: 'Vue Green'
})
```
可以通过如下方式，添加新的响应式属性：
```js
vm.userProfile = Object.assign({}, vm.userProfile, {
  age: 27,
  favoriteColor: 'Vue Green'
})
```

### 显示过滤/排序结果(Displaying Filtered/Sorted Results)

有时，我们想要显示一个数组过滤或排序后(filtered or sorted)的副本，而不是实际改变或重置原始数据。

在这种情况下，可以创建一个返回过滤或排序数组的computed属性。

`<li v-for="n in evenNumbers">{{ n }}</li>`

```js
data: {
  numbers: [ 1, 2, 3, 4, 5 ]
},
computed: {
  evenNumbers: function () {
    return this.numbers.filter(function (number) {
      return number % 2 === 0
    })
  }
}
```
在计算属性不适用的情况下（例如，在嵌套的 v-for 循环内），可以使用一个 method 方法.

```js
data: {
  numbers: [ 1, 2, 3, 4, 5 ]
},
methods: {
  even: function (numbers) {
    return numbers.filter(function (number) {
      return number % 2 === 0
    })
  }
}
```

### 使用 v-for 在整数值范围内迭代

v-for 也可以在整数值范围内迭代。在这种情况下，会将模板重复多次。
```html
<div>
  <span v-for="n in 10">{{ n }}</span>
</div>
```

### 在 `<template>` 上使用 v-for

类似于在 `<template>` 上使用 v-if，你还可以在 `<template>` 标签上使用 v-for，来渲染多个元素块。例如：

```html
<ul>
  <template v-for="item in items">
    <li>{{ item.msg }}</li>
    <li class="divider"></li>
  </template>
</ul>
```

### 带有 v-if 的 v-for

当它们都处于同一节点时，v-for 的优先级高于 v-if。这意味着，v-if 将分别在循环中的每次迭代上运行。
```html
<li v-for="todo in todos" v-if="!todo.isComplete">
  {{ todo }}
</li>
```

如果你的意图与此相反，是根据条件跳过执行循环，可以将 v-if 放置于包裹元素上（或放置于 <template> 上）
```
<ul v-if="todos.length">
  <li v-for="todo in todos">
    {{ todo }}
  </li>
</ul>
<p v-else>No todos left!</p>
```

### 使用 v-for 遍历组件

在 2.2.0+ 版本，当对组件使用 v-for 时，必须设置 key 属性。

然而，这里无法自动向组件中传入数据，这是因为组件有自己的独立作用域。为了将组件外部的迭代数据传入组件，我们还需要额外使用 props：

```html
<my-component
  v-for="(item, index) in items"
  v-bind:item="item"
  v-bind:index="index"
  v-bind:key="item.id"
></my-component>
```

没有通过 v-for 将 item 自动注入到组件中的原因是，一旦自动注入，就会使得组件与 v-for 指令的运行方式紧密耦合(tightly coupled)在一起。通过显式声明组件数据来源，可以使得组件可重用于其他场景。

一个 todo list 的完整示例：
```html
<div id="todo-list-example">
  <input
    v-model="newTodoText"
    v-on:keyup.enter="addNewTodo"
    placeholder="Add a todo"
  >
  <ul>
    <li
      is="todo-item"
      v-for="(todo, index) in todos"
      v-bind:key="todo.id"
      v-bind:title="todo.title"
      v-on:remove="todos.splice(index, 1)"
    ></li>
  </ul>
</div>
```

注意 is="todo-item" 属性。这在 DOM 模板中是必需的，因为在 <ul> 中，只有 <li> 是有效元素。这与调用 <todo-item> 的实际结果相同，但是却可以解决浏览器潜在的解析错误。